In [34]:
import numpy as np
import pandas as pd
from collections import Counter
#import joblib
%pylab inline
plt.rcParams['figure.figsize'] = 8, 6

Populating the interactive namespace from numpy and matplotlib


In [85]:
# # Intel MKL locally, but Atlas in production
# from numpy.distutils.system_info import get_info
# print(get_info('blas_opt'))
# print(get_info('lapack_opt'))

### Data Description

Data is given for each store and we are tasked with predicting the department-wide sales for each store.

**stores.csv**

This file contains anonymized information about the 45 stores, indicating the type and size of store.


**Train.csv**

- Store - the store number
- Dept - the department number
- Date - the week
- Weekly_Sales -  sales for the given department in the given store
- IsHoliday - whether the week is a special holiday week

**features.csv**

- Store - the store number
- Date - the week
- Temperature - average temperature in the region
- Fuel_Price - cost of fuel in the region
- MarkDown1-5 - anonymized data related to promotional markdowns that Walmart is running. MarkDown data is only available after Nov 2011, and is not available for all stores all the time. Any missing value is marked with an NA.
- CPI - the consumer price index
- Unemployment - the unemployment rate
- IsHoliday - whether the week is a special holiday week

Looks like we'll have to join on store and date.

## Loading

Load the data utilizing Pandas' .csv reader as it is significantly faster than NumPy's costly (due to intermediate python object steps) loadtxt. 

In [35]:
X_traindf = pd.read_table('train.csv', sep=',', warn_bad_lines=True, error_bad_lines=True)
X_testdf = pd.read_table('test.csv', sep=',', warn_bad_lines=True, error_bad_lines=True) 
stores = pd.read_table('stores.csv', sep=',', warn_bad_lines=True, error_bad_lines=True) 
X_addtl_feat = pd.read_table('features.csv', sep=',', warn_bad_lines=True, error_bad_lines=True) 

print(X_traindf.shape, X_testdf.shape, X_addtl_feat.shape, stores.shape)


(421570, 5) (115064, 4) (8190, 12) (45, 3)


Address missing store and departments between Train and Test? 
e.g. 99, stores 5,9,10 and 25 Perhaps mixed dummy creation addresses this.

In [36]:
# Check Weekly_sales for negative and zero sales.

from collections import Counter
couter = Counter(np.sign(X_traindf['Weekly_Sales']).astype(int))
print(couter)
# print np.where(X_traindf.Weekly_Sales.isnull())

Counter({1: 420212, -1: 1285, 0: 73})


## Imputation:

Treat zeros in the `Weekly_Sales` column as missing since the probability of merchandise returns equalling merchandise purchases for any given week is very low.

In [37]:
for index in np.where(X_traindf.Weekly_Sales == 0)[0]:
    X_traindf['Weekly_Sales'][index] = X_traindf.Weekly_Sales.median()

C:\Users\xiezhijun\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [38]:
X_traindf['Weekly_Sales'].fillna(value = X_traindf.Weekly_Sales.median(),inplace = True)

print(Counter(np.sign(X_traindf['Weekly_Sales']).astype(int)))

Counter({1: 420285, -1: 1285})


Replace NaN values in the `MarkDown` columns with zeros.

In [39]:
#X_addtl_feat = X_addtl_feat.replace([np.inf, -np.inf], np.nan)

for column in ('MarkDown%s' % i for i in range(1,6)):
    X_addtl_feat[column].fillna(0, inplace=True)
    
print(Counter(np.sign(X_addtl_feat['MarkDown1']).astype(int)))

Counter({0: 4158, 1: 4028, -1: 4})


Fill `CPI` and `Unemployment` with column median.

In [40]:
X_addtl_feat['CPI'].fillna(value = X_addtl_feat.CPI.median(),inplace = True)
X_addtl_feat['Unemployment'].fillna(value = X_addtl_feat.Unemployment.median(), inplace = True)

##Joining

Store and Date will be safe to join on as the formatting looks to be consistent. If the data were large it would have necessitated making the analysis I/O bound rather than memory and we would have loaded in in a database or used a chunking method with HDF5. Our simple left join would have looked something like this in SQL:

```SQL
Select *
FROM Train_Xdf A
LEFT JOIN addtl_features B
ON A.Store = B.Store
AND A.Date = B.Date
```

Join the *features* and *stores* data set with the *train* and *test* datasets.

In [41]:
X_traindf = pd.merge(X_traindf, X_addtl_feat, how='left', on=['Store', 'Date'])
X_traindf = pd.merge(X_traindf, stores, how='left', on='Store')

X_testdf = pd.merge(X_testdf, X_addtl_feat, how='left', on=['Store', 'Date'])
X_testdf = pd.merge(X_testdf, stores, how='left', on='Store')
X_testdf.head()
# X_traindf.sort(['Store', 'Date'], ascending=[1, 2]).head(1)


,Store,Dept,Date,IsHoliday_x,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday_y,Type,Size
0,1,1,2012-11-02,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,151315
1,1,1,2012-11-09,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,151315
2,1,1,2012-11-16,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,151315
3,1,1,2012-11-23,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,151315
4,1,1,2012-11-30,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,151315


##Encoding

_Markdown_ columns are not a result of binary encoding. The attached description does not clarify the values, but we'll have to assume it's a real scalar and not a categorical encoding based on all the unique values.

The _IsHoliday_  and _Type_ columns are categorical and need to be binary-encoded, since we haven't got to NumPy yet we can utilize Pandas' *get_dummies* method.

**Date**

The Date column can be binarily encoded to create the following features:

- <s>Hour of the day (24 boolean features)</s><br>
- <s>Day of the week (7 boolean features)</s> <font color = 'red'> Only provided the first day of every week</font><br>
- <s>Day of the month (up to 31 boolean features)</s><font color = 'red'> Ideal to specify 1-4 for week of month</font><br>
- Month of the year (12 boolean features)<br>
- Year (as many boolean features as they are different years in your dataset)<br>

These features will enable us to identify linear dependencies on periodic events on typical time cycles. We can also create one contiunous feature derived from POSIX time.

<br>
<font color = "dodgerblue">Concatenate `traindf` and `testdf` before creating dummy features incase some stores/dates/departments don't exist in each others set. Split by index or 0 sales value later:</font>

http://nbviewer.ipython.org/github/herrfz/dataanalysis/blob/master/assignment2/samsung_data_prediction_submitted.ipynb

In [42]:
X_testdf["Weekly_Sales"] = 0

# Reorder testdf to match 'Weekly Sales' column position of traindf. 
# A better way to do this would be to pd.concat reordered slices of test together
X_testdf = X_testdf.ix[:, ['Store','Dept','Date','Weekly_Sales','IsHoliday_x','Temperature',
                 'Fuel_Price','MarkDown1','MarkDown2','MarkDown3',
                 'MarkDown4','MarkDown5','CPI','Unemployment','IsHoliday_y',
                 'Type','Size']]
X_testdf.head()

C:\Users\xiezhijun\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


,Store,Dept,Date,Weekly_Sales,IsHoliday_x,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday_y,Type,Size
0,1,1,2012-11-02,0,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,151315
1,1,1,2012-11-09,0,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,151315
2,1,1,2012-11-16,0,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,151315
3,1,1,2012-11-23,0,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,151315
4,1,1,2012-11-30,0,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,151315


In [43]:
def dfDummies_concat(dfx): 
    ''' Create and concatenate named non-default dummy variables for identified columns'''
    
    holidayDummies = pd.get_dummies(dfx['IsHoliday_x'])
    holidayDummies.columns = ['IsHolidayF','IsHolidayT']
    dfx.drop('IsHoliday_y', 1, inplace=True)
    dfx.drop('IsHoliday_x', 1, inplace=True)
    
    dfx['Type'] = 'Type_' + dfx['Type'].astype(str)
    typeDummies = pd.get_dummies(dfx['Type'])
    typeDummies.columns = ['TypeA','TypeB', 'TypeC']
    dfx.drop('Type', 1, inplace=True)
    
    dfx['Store'] = 'Store_' + dfx['Store'].astype(str)
    storeDummies = pd.get_dummies(dfx['Store'])
    dfx.drop('Store', 1, inplace=True)
    
    dfx['Dept'] = 'dept_' + dfx['Dept'].astype(str)
    deptDummies = pd.get_dummies(dfx['Dept'])
    dfx.drop('Dept', 1, inplace=True)

    # There HAS to be a faster way than this lambda f(x)
    dateSplit = dfx['Date'].apply(lambda x: pd.Series(x.split('-')))
    dateSplit.columns  = ['year','month','day']
    dateSplit['year']  = 'year_' + dateSplit['year'].astype(str)
    dateSplit['month'] = 'month_' + dateSplit['month'].astype(str)
    dateSplit['day']   = 'day_' + dateSplit['day'].astype(str)
    
    dfx.drop('Date',1,inplace=True)
    

    yearDummies  = pd.get_dummies(dateSplit['year'])
    monthDummies = pd.get_dummies(dateSplit['month'])
    dayDummies   = pd.get_dummies(dateSplit['day'])

  
    df_concat = pd.concat( [dfx, holidayDummies, typeDummies, 
                            storeDummies, deptDummies, yearDummies,
                            monthDummies, dayDummies],
                          join='outer', axis=1, ignore_index=False)

    return df_concat

In [44]:
dfConcat = pd.concat( [X_traindf, X_testdf], join = 'outer', axis=0 , ignore_index = False )
dfConcat.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday_x,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday_y,Type,Size
0,1,1,2010-02-05,24924.50,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,151315
1,1,1,2010-02-12,46039.49,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,151315
2,1,1,2010-02-19,41595.55,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,151315
3,1,1,2010-02-26,19403.54,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,151315
4,1,1,2010-03-05,21827.90,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,151315


In [45]:
# %load_ext
# %lprun
dfConcat = dfDummies_concat(dfConcat)
print(dfConcat.shape)

# X_traindf = dfDummies_concat(X_traindf)
# X_testdf = dfDummies_concat(X_testdf)

(536634, 189)


In [46]:
dfConcat.head()

,Weekly_Sales,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,...,day_22,day_23,day_24,day_25,day_26,day_27,day_28,day_29,day_30,day_31
0,24924.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
1,46039.49,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
2,41595.55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
3,19403.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,1,0,0,0,0,0
4,21827.90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0


Split dfConcat back into X_traindf and X_testdf.

In [47]:
X_traindf = dfConcat[:421570]  
X_testdf = dfConcat[421570:]  

print(X_traindf.shape, X_testdf.shape)

(421570, 189) (115064, 189)


Create X and y NumPy arrays:

In [48]:
X = X_traindf.values[:,1:]
y = X_traindf['Weekly_Sales'].values

toPredict = X_testdf.values[:,1:] # drop empty sales col

print('X Shape:' + str(X.shape)) 
print('y Shape:' + str(y.shape))
print('toPredict Shape: ' + str(toPredict.shape)) 

# import joblib
# joblib.dump(X, 'X.pkl', compress=0, cache_size=100)

X Shape:(421570, 188)
y Shape:(421570,)
toPredict Shape: (115064, 188)


##Cross Validation splits
Create train and test data splits for cross validation.

In [49]:
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

print('X_train Shape: ' + str(X_train.shape)) 
print('y_train Shape: ' + str(y_train.shape))

X_train Shape: (316177, 188)
y_train Shape: (316177,)


##Scaling
Address zeros and negatives in `Weekly_Sales` then center all non-dummy features to the mean and component wise scale to unit variance. <font color = "red">Do not scale dummies! </font>

In [51]:
# BEFORE
# np.bincount(np.sign(y_train).astype(int))
from collections import Counter
print(Counter(np.sign(y_train).astype(int)))

Counter({1: 315219, -1: 958})


In [53]:
y_train.head()

AttributeError: 'numpy.ndarray' object has no attribute 'head'

This should be the same as scaling before creating the train and test splits since a fit on X_train and y_train are being used to transform other splits.

In [52]:
from sklearn.preprocessing import StandardScaler

# XScaler = StandardScaler().fit(X[:,0:10]) # Try fitting og X instead X_train
XTrainScaler = StandardScaler().fit(X_train[:,0:10]) 
yScaler = StandardScaler().fit(y_train) 
# toPredictScaler = StandardScaler().fit(toPredict[:,0:10]) 

# Scale Train w/ X_Train

X_train[:, 0:10] = XTrainScaler.transform(X_train[:,0:10])
# y_train = yScaler.transform(y_train)
# y_train = np.log(y_train + 1 - y_train.min())
y_train =  log1p(y_train- y_train.min()) 

# Scale Test w/ X_Train

X_test[:,0:10] = XTrainScaler.transform(X_test[:,0:10])
# y_test = yScaler.transform(y_test)
# y_test =  np.log(y_test + 1 - y_test.min())  # Should it be y_trains min and not y_test min? Not the case with X.
y_test =  log1p(y_test - y_test.min()) 

# toPredict does not come from the same X as X_train & X_test do, is it logical to transform with XTrainScaler?
# toPredict[:,0:10 ]= XTrainScaler.transform(toPredict[:,0:10])
# toPredict[:,0:10 ]= XScaler.transform(toPredict[:,0:10])

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [107]:
# AFTER
Counter(np.sign(y_train).astype(int))

Counter({1: 316176, 0: 1})

In [111]:
np.where(y_train==0), y_train[300100:300150 ]

((array([300132]),),
 array([  9.57061353,   9.72214012,   8.56073724,  10.11692212,
         11.22121194,  10.52838614,  10.8235135 ,   8.55351355,
         11.22151846,   9.35209685,   9.3580377 ,   9.88105583,
          9.48319405,   8.60317637,   8.52415689,   8.58802624,
          8.54164187,   9.85464914,   9.41182955,   9.45255059,
          9.05563554,   8.86294651,  10.65869926,   8.51577419,
          8.51616066,   9.17038363,  11.05870274,   9.21214175,
         10.5360383 ,   9.50304207,  11.080059  ,   9.43457293,
          0.        ,   9.57213429,   9.61281769,   9.37953194,
         10.10675977,   9.79129822,   8.56249155,   9.74193169,
          9.29717382,  11.13505636,   9.20380808,   8.8645333 ,
         10.08356075,   8.81884326,   8.88780181,   9.2350603 ,
          8.85075262,  10.20708456]))

##Random Forests
<font color="red">Severe bottleneck!</font>
Olivier Grisel's joblib is a nice sklearn drop-in as RandomForest is trivially parallelizable since each processor can generate forests independently:

In [65]:
# %%time

# from sklearn.ensemble import RandomForestRegressor

# rf = RandomForestRegressor(n_jobs= -1)

# # (n_estimators=10, criterion='mse', max_depth=None,
# #                            min_samples_split=2, min_samples_leaf=1, max_features='auto',
# #                            bootstrap=True, oob_score=False, n_jobs=-1, random_state=None,
# #                            verbose=0, min_density=None, compute_importances=None)

# rf.fit(X_train,y_train)

# import joblib
# joblib.dump(rf, 'rf_pickle/rf.pkl', compress=0, cache_size=100)



In [66]:
# rf.score(X_test,y_test)

### SGD

Well start with a lineal model such as SGDRegressor. This regressor attempts to find a hyperplane that minimizes a certain loss function (the sum of squared distances from each instance to the hyperplane) using Stochastic Gradient Descent to find the minimum.

In [112]:
# Check is features have been scaled appropriately
pd.DataFrame(X_train[0:1])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,
0,-0.955209,1.489699,0.470602,0.661288,-0.083006,-0.075413,-0.146441,1.100887,-0.442331,-0.139616,1,0,0,1,0,0,0,0,0,0,...


The validation set is used for model selection, the test set for final model (the model which was selected by selection process) prediction error.

In [115]:
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import KFold

def trainEval(clf, X_train, y_train):    
    clf.fit(X_train, y_train)
    print "Coefficient of determination on training set:", clf.score(X_train, y_train)
    scores = cross_val_score(clf, X_train, y_train, n_jobs =-1) #cv=cv
    print "Average coefficient of determination using 3-fold crossvalidation:",np.mean(scores)
    test_scores = cross_val_score(clf, X_test, y_test, n_jobs =-1) #cv=cv
    print "Score clf on X_test/y_test:", np.mean(test_scores)

In [114]:
from sklearn import linear_model
clf_sgd = linear_model.SGDRegressor(verbose=0,loss='squared_loss', penalty=None, shuffle=True, random_state=None)
trainEval(clf_sgd,X_train,y_train)

Coefficient of determination on training set: 0.769838314209
Average coefficient of determination using 3-fold crossvalidation: 0.768144896161
Score clf on X_test/y_test: 0.748322735946


In [116]:
clf_sgd1 = linear_model.SGDRegressor(loss='squared_loss', penalty='l2', shuffle=True, random_state=None)
trainEval(clf_sgd1,X_train,y_train)

Coefficient of determination on training set: 0.769050788749
Average coefficient of determination using 3-fold crossvalidation: 0.767874629816
Score clf on X_test/y_test: 0.748130106685


In [117]:
clf_sgd2 = linear_model.SGDRegressor(loss='squared_loss', penalty='l1', shuffle=True, random_state=None)
trainEval(clf_sgd2,X_train,y_train)

Coefficient of determination on training set: 0.769124036479
Average coefficient of determination using 3-fold crossvalidation: 0.767565505272
Score clf on X_test/y_test: 0.747832060756


In [118]:
clf_sgd3 = linear_model.SGDRegressor(loss='squared_loss', penalty='elasticnet', shuffle=True, random_state=None)
trainEval(clf_sgd3,X_train,y_train)

Coefficient of determination on training set: 0.769313174026
Average coefficient of determination using 3-fold crossvalidation: 0.767367899591
Score clf on X_test/y_test: 0.747641425828


In [119]:
clf_ridge = linear_model.Ridge()
trainEval(clf_ridge,X_train,y_train)

Coefficient of determination on training set: 0.772695558119
Average coefficient of determination using 3-fold crossvalidation: 0.772388495731
Score clf on X_test/y_test: 0.765669101638


Using X_scaler:
clf_ridge: 0.665545556463

Using y_train for both y log transforms:
clf_ridge: 0.752506048458 

Using y_train and y_test for respective log transforms:
clf_ridge:  0.768398420012

In [121]:
# from sklearn import svm
# clf_svr = svm.SVR()
# trainEval(clf_svr,X_train[0:20000],y_train[0:20000])

Coefficient of determination on training set: 0.763939276859
Average coefficient of determination using 3-fold crossvalidation: 0.73263523501
Score clf on X_test/y_test: 0.76200253459


In [36]:
# %%time
# from sklearn import ensemble
# clf_et=ensemble.ExtraTreesRegressor(n_estimators=10, n_jobs = -1)
# trainEval(clf_et,X_train,y_train)

In [35]:
# print sort(zip(clf_et.feature_importances_,list(X_traindf.columns)),axis=0)

### Random Search functions:

In [18]:
# from scipy.stats import randint as sp_randint
# from time import time
# from operator import itemgetter
# from sklearn.grid_search import RandomizedSearchCV

# # Util function to report best scores
# def grscoreReport(grid_scores, n_top=3):
#     top_scores = sorted(grid_scores, key=itemgetter(1), reverse=True)[:n_top]
#     for i, score in enumerate(top_scores):
#         print("Model with rank: {0}".format(i + 1))
#         print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
#               score.mean_validation_score,
#               np.std(score.cv_validation_scores)))
#         print("Parameters: {0}".format(score.parameters))
#         print("")

In [ ]:
# # specify parameters and distributions to sample from
# param_dist = {"max_depth": [3, None],
#               "max_features": [sp_randint(1, 11),'auto'],
#               "min_samples_split": sp_randint(1, 11),
#               "min_samples_leaf": sp_randint(1, 11),
#               "bootstrap": [True, False],
#               "criterion": ['mse']}

# # run randomized search
# start = time()

# n_iter_search = 10

# random_search = RandomizedSearchCV(rf, param_distributions=param_dist, 
#                                    n_iter=n_iter_search, n_jobs = -1)  

# random_search.fit(X_train, y_train)

# print("RandomizedSearchCV took %.2f minutes for %d candidates"
#       " parameter settings." % ((time() - start), n_iter_search))

# grscoreReport(random_search.grid_scores_)

In [88]:
# from sklearn.dummy import DummyRegressor 
# reg = DummyRegressor()
# reg.fit(X_train, y_train)
# print reg.score(X_test, y_test)

#### Writing submission:

Need to drop the 'Weekly Sales' zeros column from X_testdf before training for submission:

In [80]:
test_csv = pd.read_table('data/test.csv', sep=',', warn_bad_lines=True, error_bad_lines=True) 

print test_csv.shape 
print 'With sales columns: ' + str(X_traindf.shape) + str(X_testdf.shape)
print 'Without: ' + str(X_train.shape) + str(X_test.shape)
print 'toPredict (test.csv): ' + str(toPredict.shape)

(115064, 4)
With sales columns: (421570, 189)(115064, 189)
Without: (316177, 188)(105393, 188)
toPredict (test.csv): (115064, 188)


In [81]:
%%time
import time
import datetime

def CurrHr():
    epoch_time = int(time.time())
    epoch_time = epoch_time # Round to hour
    current_hour = datetime.datetime.fromtimestamp(epoch_time).strftime('%Y%m%d%H%M%S')
    return current_hour


subResults = pd.DataFrame(clf_ridge.predict(toPredict), columns = ['Weekly_Sales'])
# subResults =  (exp(subResults)-1)

subLabels = pd.DataFrame(test_csv.Store.astype(str) + '_' + 
             test_csv.Dept.astype(str) + '_' +
             test_csv.Date.astype(str),columns = ['Id'])
subResults.shape , subLabels.shape

CPU times: user 1.19 s, sys: 15.8 ms, total: 1.2 s
Wall time: 834 ms


In [83]:
print (exp(subResults)-1)[:5]
print subResults[:5]

   Weekly_Sales
0      0.813753
1      1.227697
2      0.625241
3      0.970099
4      0.819807

[5 rows x 1 columns]
   Weekly_Sales
0      0.595398
1      0.800968
2      0.485656
3      0.678084
4      0.598730

[5 rows x 1 columns]


In [84]:
# Dropping column names for unknown reason on concat
dfForCSV = pd.concat([subLabels,subResults], axis=1, ignore_index=True,)
dfForCSV.columns = ['Id', 'Weekly_Sales']
dfForCSV.to_csv(CurrHr() + '.csv', index = False, header = True)

In [44]:
# from ggplot import *
# ggplot(diamonds, aes('carat', 'price')) + \
# geom_point(alpha=1/20.) + \
# ylim(0, 20000)


# ggplot(X_traindf, aes(x=Month, y=Monthly_Sales)) + geom_bar(stat="identity")